<a href="https://colab.research.google.com/github/jscscloris/2048-api/blob/master/2048.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

初始数据

In [ ]:
%cd /content/driver/Shared\ drives/jscscloris@gmail.com/2048-api
from game2048.game import Game
from game2048.displays import Display, IPythonDisplay
from game2048.agents import Agent, RandomAgent, ExpectiMaxAgent
import numpy as np
import json
import torch.nn as nn
from torch.utils.data import TensorDataset,DataLoader
import torch

map_table={2**i:i for i in range(1,16)}
map_table[0]=0
def grid_ohe(arr):
  ret=np.zeros(shape=(4,4,16),dtype=float)
  for r in range(4):
    for c in range(4):
      ret[r,c,map_table[arr[r,c]]]=1
  return ret
  
X_train_64=[]
Y_train_64=[]
X_train_128=[]
Y_train_128=[]
X_train_256=[]
Y_train_256=[]
X_train_512=[]
Y_train_512=[]
X_train_1024=[]
Y_train_1024=[]
iters=1000

for i in range(iters):
  game = Game(4, score_to_win=2048, random=False)
  agent = ExpectiMaxAgent(game)
  while game.end==0:  
    bd=np.array(game.board)
    bd_ohe=grid_ohe(bd)
    bd_ohe=np.swapaxes(bd_ohe,0,2)
    score=game.score
    direction = agent.step()
    if score<=64:
      X_train_64.append(bd_ohe)
      Y_train_64.append([direction])
    elif score==128:
      X_train_128.append(bd_ohe)
      Y_train_128.append([direction])
    elif score==256:
      X_train_256.append(bd_ohe)
      Y_train_256.append([direction])  
    elif score==512:
      X_train_512.append(bd_ohe)
      Y_train_512.append([direction]) 
    else:
      X_train_1024.append(bd_ohe)
      Y_train_1024.append([direction])
    game.move(direction) 
  print('iter:',i)

print('length of dataset64:',len(X_train_64))
print('length of dataset128:',len(X_train_128))
print('length of dataset256:',len(X_train_256))
print('length of dataset512:',len(X_train_512))
print('length of dataset1024:',len(X_train_1024))
X=np.array(X_train_64) 
Y=np.array(Y_train_64) 
np.save('X_train_64.npy', X)
np.save('Y_train_64.npy', Y)
X=np.array(X_train_128) 
Y=np.array(Y_train_128) 
np.save('X_train_128.npy', X)
np.save('Y_train_128.npy', Y)
X=np.array(X_train_256) 
Y=np.array(Y_train_256) 
np.save('X_train_256.npy', X)
np.save('Y_train_256.npy', Y)
X=np.array(X_train_512) 
Y=np.array(Y_train_512) 
np.save('X_train_512.npy', X)
np.save('Y_train_512.npy', Y)
X=np.array(X_train_1024) 
Y=np.array(Y_train_1024) 
np.save('X_train_1024.npy', X)
np.save('Y_train_1024.npy', Y)

训练部分

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets
from torch.autograd import Variable
from torch.utils.data import TensorDataset,DataLoader
import time
import numpy as np

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(16, 64, kernel_size=(4, 1), padding=(2, 0))
        self.conv2 = nn.Conv2d(64, 128, kernel_size=(1, 4), padding=(0, 2))
        self.conv3 = nn.Conv2d(128, 128, kernel_size=(2, 2))
        self.conv4 = nn.Conv2d(128, 128, kernel_size=(3, 3), padding=(1, 1))
        self.conv5 = nn.Conv2d(128, 128, kernel_size=(4, 4), padding=(2, 2))
        self.conv6 = nn.Conv2d(128,128,kernel_size=(2,2))
        self.fc1 = nn.Linear(128 * 4 * 4, 2048)
        self.fc2 = nn.Linear(2048, 512)
        self.fc3 = nn.Linear(512, 4)
        self.batch_norm1 = nn.BatchNorm1d(128 * 4 * 4)
        self.batch_norm2 = nn.BatchNorm1d(2048)
        self.batch_norm3 = nn.BatchNorm1d(512)

        self.initialize()

    def forward(self,x):        
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))
        x = x.view(-1, 128 * 4 * 4)
        x = self.batch_norm1(x)
        x = F.relu(self.fc1(x))
        x = self.batch_norm2(x)
        x = F.relu(self.fc2(x))
        x = self.batch_norm3(x)
        x = self.fc3(x)

        return x

    def initialize(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_uniform_(m.weight, mode='fan_in')
            elif isinstance(m, nn.Linear):
                nn.init.kaiming_uniform_(m.weight, mode='fan_in')

batch_size = 128
NUM_EPOCHS = 200

X_train=np.load('X_train_64.npy')
Y_train=np.load('Y_train_64.npy')
train_db = TensorDataset(torch.Tensor(X_train).float(), torch.squeeze(torch.Tensor(Y_train)))
train_loader_64 = DataLoader(train_db, batch_size=batch_size, shuffle=True)

X_train=np.load('X_train_128.npy')
Y_train=np.load('Y_train_128.npy')
train_db = TensorDataset(torch.Tensor(X_train).float(), torch.squeeze(torch.Tensor(Y_train)))
train_loader_128 = DataLoader(train_db, batch_size=batch_size, shuffle=True)

X_train=np.load('X_train_256.npy')
Y_train=np.load('Y_train_256.npy')
train_db = TensorDataset(torch.Tensor(X_train).float(), torch.squeeze(torch.Tensor(Y_train)))
train_loader_256 = DataLoader(train_db, batch_size=batch_size, shuffle=True,drop_last=True)

X_train=np.load('X_train_512.npy')
Y_train=np.load('Y_train_512.npy')
train_db = TensorDataset(torch.Tensor(X_train).float(), torch.squeeze(torch.Tensor(Y_train)))
train_loader_512 = DataLoader(train_db, batch_size=batch_size, shuffle=True,drop_last=True)

X_train=np.load('X_train_1024.npy')
Y_train=np.load('Y_train_1024.npy')
train_db = TensorDataset(torch.Tensor(X_train).float(), torch.squeeze(torch.Tensor(Y_train)))
train_loader_1024 = DataLoader(train_db, batch_size=batch_size, shuffle=True,drop_last=True)
model = Net()
model = model.cuda()
loss_func=nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

model.train()

model.load_state_dict(torch.load('64_epoch_200.pkl'))
for epoch in range(NUM_EPOCHS):
  for step, (X_train, Y_train) in enumerate(train_loader_64):
    X_train=Variable(X_train).cuda()
    Y_train=Variable(Y_train).cuda()
    optimizer.zero_grad()
    output=model(X_train)
    loss = loss_func(output,Y_train.long())    
    loss.backward()
    optimizer.step()  
  if (epoch+1)%100==0:
    print('save64',(epoch+1),loss)
    torch.save(model.state_dict(), '64_epoch_{}.pkl'.format((epoch+1)))

model.load_state_dict(torch.load('128_epoch_200.pkl'))
for epoch in range(NUM_EPOCHS):
  for step, (X_train, Y_train) in enumerate(train_loader_128):
    X_train=Variable(X_train).cuda()
    Y_train=Variable(Y_train).cuda()
    optimizer.zero_grad()
    output=model(X_train)
    loss = loss_func(output,Y_train.long())    
    loss.backward()
    optimizer.step()
  if (epoch+1)%100==0:
    print('save128',(epoch+1),loss)
    torch.save(model.state_dict(), '128_epoch_{}.pkl'.format((epoch+1)))

model.load_state_dict(torch.load('256_epoch_200.pkl'))
for epoch in range(NUM_EPOCHS):
  for step, (X_train, Y_train) in enumerate(train_loader_256):
    X_train=Variable(X_train).cuda()
    Y_train=Variable(Y_train).cuda()
    optimizer.zero_grad()
    output=model(X_train)
    loss = loss_func(output,Y_train.long())    
    loss.backward()
    optimizer.step()
  if (epoch+1)%100==0:
    print('save256',(epoch+1),loss)
    torch.save(model.state_dict(), '256_epoch_{}.pkl'.format((epoch+1)))

model.load_state_dict(torch.load('512_epoch_400.pkl'))
for epoch in range(NUM_EPOCHS):
  for step, (X_train, Y_train) in enumerate(train_loader_512):
    X_train=Variable(X_train).cuda()
    Y_train=Variable(Y_train).cuda()
    optimizer.zero_grad()
    output=model(X_train)
    loss = loss_func(output,Y_train.long())    
    loss.backward()
    optimizer.step()
  if (epoch+1)%100==0:
    print('save512',(epoch+1),loss)
    torch.save(model.state_dict(), '512_epoch_{}.pkl'.format((epoch+1)))

model.load_state_dict(torch.load('1024_epoch_200.pkl'))
for epoch in range(NUM_EPOCHS):
  for step, (X_train, Y_train) in enumerate(train_loader_1024):
    X_train=Variable(X_train).cuda()
    Y_train=Variable(Y_train).cuda()
    optimizer.zero_grad()
    output=model(X_train)
    loss = loss_func(output,Y_train.long())    
    loss.backward()
    optimizer.step()
  if (epoch+1)%100==0:
    print('save1024',(epoch+1),loss)
    torch.save(model.state_dict(), '1024_epoch_{}.pkl'.format((epoch+1)))


数据增广部分

In [ ]:
from game2048.game import Game
from game2048.displays import Display, IPythonDisplay
from game2048.agents import Agent, RandomAgent, ExpectiMaxAgent,MyOwnAgent
import numpy as np
import json
import torch.nn as nn
from torch.utils.data import TensorDataset,DataLoader
import torch

map_table={2**i:i for i in range(1,16)}
map_table[0]=0
def grid_ohe(arr):
  ret=np.zeros(shape=(4,4,16),dtype=float)
  for r in range(4):
    for c in range(4):
      ret[r,c,map_table[arr[r,c]]]=1
  return ret
  
X_train_64=[]
Y_train_64=[]
X_train_128=[]
Y_train_128=[]
X_train_256=[]
Y_train_256=[]
X_train_512=[]
Y_train_512=[]
X_train_1024=[]
Y_train_1024=[]
iters=1000

for i in range(iters):
  game = Game(4, score_to_win=2048, random=False)
  agent = MyOwnAgent(game)
  expect = ExpectiMaxAgent(game)
  while game.end==0:  
    bd=np.array(game.board)
    bd_ohe=grid_ohe(bd)
    bd_ohe=np.swapaxes(bd_ohe,0,2)
    score=game.score
    direction = expect.step()
    mo=agent.step()
    if score<=64:
      X_train_64.append(bd_ohe)
      Y_train_64.append([direction])
    elif score==128:
      X_train_128.append(bd_ohe)
      Y_train_128.append([direction])
    elif score==256:
      X_train_256.append(bd_ohe)
      Y_train_256.append([direction])  
    elif score==512:
      X_train_512.append(bd_ohe)
      Y_train_512.append([direction]) 
    else:
      X_train_1024.append(bd_ohe)
      Y_train_1024.append([direction])
    game.move(mo) 
  

print('length of dataset64:',len(X_train_64))
print('length of dataset128:',len(X_train_128))
print('length of dataset256:',len(X_train_256))
print('length of dataset512:',len(X_train_512))
print('length of dataset1024:',len(X_train_1024))

X=np.array(X_train_64) 
Y=np.array(Y_train_64) 
raw_X=np.load('X_train_64.npy')
raw_Y=np.load('Y_train_64.npy')
new_X=np.concatenate((raw_X, X), axis=0)
new_Y=np.concatenate((raw_Y, Y), axis=0)
np.save('X_train_64.npy', new_X)
np.save('Y_train_64.npy', new_Y)

X=np.array(X_train_128) 
Y=np.array(Y_train_128) 
raw_X=np.load('X_train_128.npy')
raw_Y=np.load('Y_train_128.npy')
new_X=np.concatenate((raw_X, X), axis=0)
new_Y=np.concatenate((raw_Y, Y), axis=0)
np.save('X_train_128.npy', new_X)
np.save('Y_train_128.npy', new_Y)

X=np.array(X_train_256) 
Y=np.array(Y_train_256) 
raw_X=np.load('X_train_256.npy')
raw_Y=np.load('Y_train_256.npy')
new_X=np.concatenate((raw_X, X), axis=0)
new_Y=np.concatenate((raw_Y, Y), axis=0)
np.save('X_train_256.npy', new_X)
np.save('Y_train_256.npy', new_Y)

X=np.array(X_train_512) 
Y=np.array(Y_train_512) 
raw_X=np.load('X_train_512.npy')
raw_Y=np.load('Y_train_512.npy')
new_X=np.concatenate((raw_X, X), axis=0)
new_Y=np.concatenate((raw_Y, Y), axis=0)
np.save('X_train_512.npy', new_X)
np.save('Y_train_512.npy', new_Y)

X=np.array(X_train_1024) 
Y=np.array(Y_train_1024) 
raw_X=np.load('X_train_1024.npy')
raw_Y=np.load('Y_train_1024.npy')
new_X=np.concatenate((raw_X, X), axis=0)
new_Y=np.concatenate((raw_Y, Y), axis=0)
np.save('X_train_1024.npy', new_X)
np.save('Y_train_1024.npy', new_Y)
